# Capture the Flag Miguel: Forecast de Nível do Rio Itajaí-Açu
O objetivo desse notebook é cobrir todas as etapas de desenvolvimento de um modelo de Machine Learning para previsão de nível do Rio Itajaí-Açu. Rio do Sul historicamente possui problemas de enchentes que afetam draticamente a cidade, impactando diretamente a vida da população. O modelo tentará prever o nível do rio, com uma certa precisão, baseado em dados históricos resgatados da API do [Open Meteo](https://open-meteo.com/) e também dos dados de nível do rio capturados pela [Defesa Civil de Rio do Sul](https://defesacivil.riodosul.sc.gov.br/).

Etapas:
- Coleta de dados via API e Webscraping ✅
- Análise Exploratória das Enchentes nos últimos 6 anos ⌛️
    - Relatório com indicadores, médias e gráficos sobre os acontencimentos de enchente
- Feature Engineering ⌛️
    - Criação de novas features
    - Inclusão de novas features (mais informações da API do Open Meteo)
- Treinamento do Modelo ⌛️
    - Comparação entre modelos (random forest vs xgboost vs outros)
    - Métricas de desempenho
    - Validação cruzada
    - Tuning de hiperparâmetros
    - Seleção da melhor target: water_level_next_1h, water_level_next_3h, water_level_next_6h ...
- Avaliação de performance do melhor modelo ⌛️
- Deploy via API ⌛️
- Website contendo gráfico de nível com a previsão ⌛️
    - Vide exemplo: https://miro.medium.com/v2/resize:fit:1400/1*N3rDEJAvV_wolqXFl8HtEw.png

In [1]:
import os
import sys

# add root to sys.path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('.'), '..')))

In [2]:
from data.generate_data import generate

In [3]:
train_df = generate(
    start_date="2025-03-31",
    end_date="2025-04-06",
    type="train",
    save=False
)

2025-04-13 15:32:53.665 | INFO     | data.scraping:__init__:12 - WebScraper inicializado
2025-04-13 15:32:53.669 | INFO     | data.generate_data:__init__:31 - DataGenerator initialized with output directory: /Users/matheus/Documents/projects/Enchentes/data/output
2025-04-13 15:32:53.670 | INFO     | data.generate_data:generate:170 - Generating train dataset from 2025-03-31 to 2025-04-06
2025-04-13 15:32:53.671 | INFO     | data.generate_data:_get_weather_data:44 - Fetching weather data from 2025-03-31 to 2025-04-06
2025-04-13 15:32:54.426 | INFO     | data.generate_data:_get_weather_data:48 - Weather data fetched successfully: (168, 4) rows
2025-04-13 15:32:54.426 | INFO     | data.generate_data:_get_water_level_data:65 - Scraping water level data from 2025-03-31 to 2025-04-06
2025-04-13 15:32:54.427 | INFO     | data.scraping:parse_data:24 - Iniciando análise de dados de 2025-03-31 até 2025-04-06
2025-04-13 15:32:54.427 | INFO     | data.scraping:parse_data:30 - Serão processados 2 in

Processando intervalos:   0%|          | 0/3 [00:00<?, ?it/s]

2025-04-13 15:32:54.959 | WARNING  | data.scraping:parse_data:42 - Intervalo de datas vazio encontrado, pulando
2025-04-13 15:32:54.959 | INFO     | data.scraping:parse_data:68 - Análise de dados concluída com sucesso
2025-04-13 15:32:54.963 | INFO     | data.scraping:parse_data:80 - Formato final do DataFrame: (98, 2)
2025-04-13 15:32:54.964 | INFO     | data.generate_data:_get_water_level_data:68 - Water level data scraped successfully: (98, 2) rows
2025-04-13 15:32:54.968 | INFO     | data.generate_data:_merge_datasets:106 - Datasets merged successfully: (168, 6) rows
2025-04-13 15:32:54.976 | INFO     | data.generate_data:_process_data:145 - Data processing completed: (121, 17) rows


In [4]:
train_df.head()

,time,temperature_2m,relative_humidity_2m,apparent_temperature,rain,water_level,hour,day_of_week,month,rain_24h,temperature_24h_avg,humidity_24h_avg,water_level_next_1h,water_level_next_3h,water_level_next_6h,water_level_next_12h,water_level_next_24h
23,2025-03-31 23:00:00,23.0,87,27.0,0.0,1.46,23,0,3,0.4,24.912500,80.000000,1.46,1.46,1.44,1.29,1.32
24,2025-04-01 00:00:00,22.5,90,26.0,0.0,1.46,0,1,4,0.4,24.920833,79.833333,1.45,1.44,1.44,1.32,1.33
25,2025-04-01 01:00:00,21.6,93,25.5,0.0,1.45,1,1,4,0.4,24.887500,79.833333,1.46,1.44,1.44,1.28,1.32
26,2025-04-01 02:00:00,21.1,96,24.9,0.0,1.46,2,1,4,0.4,24.837500,79.958333,1.44,1.44,1.38,1.25,1.31
27,2025-04-01 03:00:00,20.7,98,24.6,0.0,1.44,3,1,4,0.4,24.770833,80.166667,1.44,1.44,1.37,1.22,1.32


In [5]:
predict_df = generate(
    start_date="2025-04-07",
    end_date="2025-04-12",
    type="predict",
    save=False
)

2025-04-13 15:34:02.875 | INFO     | data.scraping:__init__:12 - WebScraper inicializado
2025-04-13 15:34:02.877 | INFO     | data.generate_data:__init__:31 - DataGenerator initialized with output directory: /Users/matheus/Documents/projects/Enchentes/data/output
2025-04-13 15:34:02.877 | INFO     | data.generate_data:generate:170 - Generating predict dataset from 2025-04-07 to 2025-04-12
2025-04-13 15:34:02.878 | INFO     | data.generate_data:_get_weather_data:44 - Fetching weather data from 2025-04-07 to 2025-04-12
2025-04-13 15:34:03.599 | INFO     | data.generate_data:_get_weather_data:48 - Weather data fetched successfully: (144, 4) rows
2025-04-13 15:34:03.605 | INFO     | data.generate_data:_process_data:145 - Data processing completed: (121, 11) rows


In [6]:
predict_df.head()

,time,temperature_2m,relative_humidity_2m,apparent_temperature,rain,hour,day_of_week,month,rain_24h,temperature_24h_avg,humidity_24h_avg
23,2025-04-07 23:00:00,18.5,81.0,19.5,0.0,23,0,4,1.3,18.670833,82.125000
24,2025-04-08 00:00:00,18.2,84.0,19.3,0.0,0,1,4,1.3,18.841667,81.666667
25,2025-04-08 01:00:00,18.2,86.0,19.2,0.0,1,1,4,1.3,19.016667,81.250000
26,2025-04-08 02:00:00,18.1,88.0,18.9,0.0,2,1,4,1.3,19.162500,80.916667
27,2025-04-08 03:00:00,18.1,89.0,19.0,0.0,3,1,4,1.3,19.275000,80.666667
